In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from _src import EnergyTrading

### data init

In [3]:
file_path = "ami_datas/apt_780_201809.csv"
trade = EnergyTrading(file_path, month=9)

trade.datas.head()

,name,usage (kWh)
0,601_101,287.717
1,601_102,231.085
2,601_103,198.624
3,601_104,242.755
4,601_201,200.995


### set init
- 설정값
1. season : 계절
2. step_limit : usageBorder, 거래 가능 사용량 구하기 위한
3. sum_of_supply_kwh : 거래 가능 사용량 누진단계 1단계 미만 가구들의 총 합

In [4]:
trade.set_init()

trade.datas

,name,usage (kWh)
0,601_101,287.717
1,601_102,231.085
2,601_103,198.624
3,601_104,242.755
4,601_201,200.995
...,...,...
775,610_1804,220.434
776,610_1903,283.028
777,610_1904,404.836
778,610_2003,220.235


### simulate

In [5]:
import time

start = time.process_time()
trade.simulate()
end = time.process_time()
print(end - start)

trade.result.shape

22.583385


(1095, 4)

In [138]:
def get_settlement_infos(self):
    # Original 가격 구하기
    household_count = len(self.datas)
    
    total_household_usage = self.datas['usage (kWh)'].sum()
    public_usage = total_household_usage * 0.25
    
    APT = total_household_usage + public_usage
    APT_mean = APT / household_count
    apt = Household(name='apt', kwh=APT_mean).set_rate("단일", self.season)
    
    original_households_charge = self.datas.apply(
        lambda x: Household(name=x['name'], kwh=x['usage (kWh)'])\
            .set_rate("단일", trade.season).elec_rate, axis=1)\
                .values
    self.datas['전력량요금'] = original_households_charge
    whole_apt_charge = apt.elec_rate * household_count
    households_charge = original_households_charge.sum()
    public_charge = whole_apt_charge - households_charge
    self.datas['공용부요금'] = public_charge / household_count
    
    # 거래 정보
    result_df = pd.DataFrame(self.result, columns=["index", "거래수량", "거래액수(1kWh)", "이익"])
    total_trade_energy = result_df['거래수량'].sum()
    total_sales = (result_df['거래액수(1kWh)'] * result_df['거래수량']).sum()
    # 판매자들이 1kWh당 받을 수 있는 가격
    pri_total_sales = total_sales / total_trade_energy
    trade_result = pd.DataFrame(np.append(self.datas[['name', 'usage (kWh)']].values,np.zeros((self.datas.index.size, 4)), axis=1),
                               columns=['name', 'usage (kWh)', '구매수량', '구매가격', '판매수량', '판매가격'])
    
    result_df['총 거래액수'] = result_df['거래액수(1kWh)'] * result_df['거래수량']
    group_result = result_df.groupby('index').sum()[['거래수량', '총 거래액수']]

    # 구매자 정보
    trade_result.loc[group_result.index.values, '구매수량'] = group_result['거래수량']
    trade_result.loc[group_result.index.values, '구매가격'] = group_result['총 거래액수']

    # 판매자 정보
    seller_datas = trade_result[trade_result['usage (kWh)'] < self.step_limit[0]].copy()
    trade_result.loc[seller_datas.index, '판매수량'] = self.step_limit[0] - seller_datas['usage (kWh)']
    trade_result.loc[seller_datas.index, '판매가격'] = trade_result.loc[seller_datas.index, '판매수량'] * pri_total_sales

    trade_result
    
    return trade_result

In [139]:
trade_result = get_settlement_infos(trade)[10:20]

,name,usage (kWh),구매수량,구매가격,판매수량,판매가격
10,601_303,163.984,0.0,0.0,36.016,5188.429478
11,601_304,348.426,15.0,2098.389407,0.0,0.0
12,601_401,232.431,0.0,0.0,0.0,0.0
13,601_402,208.679,0.0,0.0,0.0,0.0
14,601_403,196.948,0.0,0.0,3.052,439.668113
15,601_404,304.936,0.0,0.0,0.0,0.0
16,601_501,270.566,0.0,0.0,0.0,0.0
17,601_502,210.71,0.0,0.0,0.0,0.0
18,601_503,297.597,0.0,0.0,0.0,0.0
19,601_504,425.619,90.0,12766.581811,0.0,0.0


In [168]:
# 사용량 : 판매한 애들은 올라가고, 구매한 애들은 내려감
simulation_datas = self.datas[['name',
                              'usage (kWh)']].copy()

simulation_datas['usage (kWh)'] += trade_result['판매수량']
simulation_datas['usage (kWh)'] -= trade_result['구매수량']

mod_households_charge = simulation_datas.apply(
        lambda x: Household(name=x['name'], kwh=x['usage (kWh)'])\
            .set_rate("단일", trade.season).elec_rate, axis=1)\
                .values
simulation_datas['전력량 요금'] = self.datas['전력량요금']
simulation_datas['거래 전력량요금(거래가격 적용 X)'] = mod_households_charge
simulation_datas['거래 전력량요금(거래가격 적용 O)'] = mod_households_charge

# 거래가격 적용 : 판매한 애들은 내려가고, 구매한 애들은 올라감
simulation_datas['거래 전력량요금(거래가격 적용 O)'] -= trade_result['판매가격'].values
simulation_datas['거래 전력량요금(거래가격 적용 O)'] += trade_result['구매가격'].values

simulation_datas['차이'] = simulation_datas['전력량 요금'] - simulation_datas['거래 전력량요금(거래가격 적용 O)']

simulation_datas.loc[self.datas.sort_values(by=['usage (kWh)'], ascending=False).index]

,name,usage (kWh),전력량 요금,거래 전력량요금(거래가격 적용 X),거래 전력량요금(거래가격 적용 O),차이
707,610_104,332.716,98447.9896,33545.4868,92045.512254,6402.477346
475,607_1003,332.352,58357.3312,33493.6896,54117.085633,4240.245567
705,610_102,335.177,56846.2762,33895.6871,52938.301621,3907.974579
60,601_1601,334.868,55728.2008,33851.7164,52072.08286,3656.11794
426,606_1802,334.642,55680.6052,33819.5566,52035.062339,3645.542861
...,...,...,...,...,...,...
244,604_902,200.0,4145.7747,14660.0000,-6003.969145,10149.743845
143,602_1904,200.0,3994.2636,14660.0000,-6301.739128,10296.002728
133,602_1702,200.0,1125.8880,14660.0000,-11939.056496,13064.944496
22,601_603,200.0,818.1013,14660.0000,-12543.960299,13362.061599


In [167]:
trade_result.loc[self.datas.sort_values(by=['usage (kWh)'], ascending=False).index]['구매가격'].values

array([58500.02545415372, 20623.396032561115, 19042.614521189655,
       18220.366459676276, 18215.50573946216, 17417.645783405493,
       17405.494558892555, 17352.214227285498, 16620.12628307953,
       16606.204366054255, 16592.419954026725, 16588.04674019236,
       15793.03987927676, 15787.561089021767, 15060.170098754093,
       14230.500521019663, 13504.320042756674, 13471.755863691513,
       12766.581810580019, 12743.659540988745, 12741.0139335201,
       11987.147668184876, 11268.199982778782, 10551.348385001356,
       10544.85488396857, 10540.801918911473, 10533.590655578642,
       9827.040146741576, 9826.617196516734, 9823.325457751589,
       9821.969315716105, 9821.22884740435, 9820.108842828526,
       9115.3591191216, 9113.839589271107, 9113.66480111058,
       9113.022513726779, 8412.837416264578, 8411.482342412171,
       8411.27008520114, 8410.046092496392, 7710.334573484921,
       7707.696189803702, 7707.62704310721, 7007.933039709342,
       7007.55727738205, 70

In [157]:
mod_households_charge

array([27142.1291, 19083.3955, 14660.    , 20744.0365, 14801.5885,
       22916.5306, 19906.1741, 28952.0428, 32031.5571, 14660.    ,
       14660.    , 33646.5198, 19274.9313, 15895.0217, 14660.    ,
       29592.3928, 24701.5418, 16184.033 , 28548.0531, 33958.5837,
       23452.2901, 21132.2309, 14660.    , 18302.1685, 14660.    ,
       25459.2893, 33418.5552, 14660.    , 23275.8381, 18698.0471,
       29483.5333, 19297.9839, 15390.2836, 34026.4608, 14660.    ,
       19910.0162, 33454.4148, 20122.4701, 31919.4247, 14660.    ,
       21828.0779, 24893.6468, 32756.7179, 16593.1455, 25422.5759,
       33327.9101, 24344.6534, 28266.2991, 33849.0127, 14660.    ,
       21955.2941, 23404.6196, 14660.    , 14660.    , 14660.    ,
       21660.1639, 33379.2804, 33765.6249, 15210.701 , 20395.4015,
       33851.7164, 14660.    , 33626.0286, 21769.1657, 19718.765 ,
       23264.7387, 28358.6518, 33777.1512, 17320.0139, 18525.1526,
       25858.7254, 14660.    , 14660.    , 14660.    , 18160.1

### Get Settlement

In [6]:
import pandas as pd

# result 의미
# 이익이 minus면 손해 구조


,index,거래수량,거래액수(1kWh),이익
0,707.0,5.000,231.212020,-103.060100
1,707.0,5.000,229.487020,-94.435100
2,707.0,5.000,227.762020,-85.810100
3,707.0,5.000,226.037020,-77.185100
4,707.0,5.000,224.312020,-68.560100
...,...,...,...,...
1090,679.0,5.000,139.767332,12.663339
1091,738.0,5.000,139.766774,12.666128
1092,45.0,5.000,139.766719,12.666407
1093,98.0,5.000,139.765360,12.673200


In [7]:
from _src.EnergyTrading.model import Household

hh = Household(name='test', kwh=662.716).set_rate("단일", "기타")

In [8]:
hh.elec_rate

98447.9896

In [ ]:
import numpy as np
def elec_rate(t):
    step_limits = np.array([200, 200, float("inf")])
    steps = np.array([0, 200, 400])
    
    _1 = t - steps
    _2 = _1[_1 > 0]
    
    size = _2.size
    _3 = np.where(_2 < step_limits[:size], _2, step_limits[:size])
    print(_3)
    
elec_rate(460)